In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from tqdm import tqdm
import pandas as pd
import numpy as np
from pathlib import Path

# Load annotation versions

In [3]:
def set_additional_fields(frame):
    
    frame["width"] = [vector["x2"] - vector["x1"] for vector in frame['vector']]
    frame["height"] = [vector["y2"] - vector["y1"] for vector in frame['vector']]
    frame["scales"] = frame["width"] / frame["height"]
    
    return frame

In [9]:
species = "Human"

v0 = pd.read_pickle(f"EIPH_{species}_SDATA_Inference_Annotations.pkl")
v0 = set_additional_fields(v0)
print(f"Inference: {len(v0)}")

v_cluster = pd.read_pickle(f"EIPH_{species}_SDATA_Cluster_Annotations.pkl")
v_cluster = set_additional_fields(v_cluster)
print(f"Cluster: {len(v_cluster)}")

v_screening = pd.read_pickle(f"EIPH_{species}_SDATA_Screening_Annotations.pkl")
v_screening = set_additional_fields(v_screening)
print(f"Screening: {len(v_screening)}")

v_density = pd.read_pickle(f"EIPH_{species}_SDATA_Density_Annotations.pkl")
print(f"Density: {len(v_density)}")

Inference: 168411
Cluster: 128012
Screening: 54580
Density: 53864


## Filter images

In [10]:
screened = [
    '10052_19 humane BAL Berliner Blau-001.svs',
    '10080_19 humane BAL Berliner Blau-001.svs',
    '10120_19 humane BAL Berliner Blau.svs',
    '10227_19 humane BAL Berliner Blau.svs',
    '10672_19 humane BAL Berliner Blau.svs',
    '10672_19 humane BAL Turnbull Blau.svs',
    '10835_19 humane BAL Berliner Blau.svs',
    '10835_19 humane BAL Turnbull Blau.svs',
    '11323_19 humane BAL Berliner Blau.svs',
    '11480_19 humane BAL Berliner Blau.svs',
    '2707_20 TB BAL Human neu.svs',
    '2702_20 BB Human BAL-001.svs', 
    
    '10052_19 humane BAL Berliner Blau-001.tiff',
    '10080_19 humane BAL Berliner Blau-001.tiff',
    '10120_19 humane BAL Berliner Blau.tiff',
    '10227_19 humane BAL Berliner Blau.tiff',
    '10672_19 humane BAL Berliner Blau.tiff',
    '10672_19 humane BAL Turnbull Blau.tiff',
    '10835_19 humane BAL Berliner Blau.tiff',
    '10835_19 humane BAL Turnbull Blau.tiff',
    '11323_19 humane BAL Berliner Blau.tiff',
    '11480_19 humane BAL Berliner Blau.tiff',
    '2707_20 TB BAL Human neu.tiff',
    '2702_20 BB Human BAL-001.tiff', 
]

v0 = v0[v0["image_name"].isin(screened)]
v_screening = v_screening[v_screening["image_name"].isin(screened)]
v_density = v_density[v_density["image_name"].isin(screened)]

In [11]:
v_screening[v_screening["image_name"].isin(screened)]

,id,image_id,image_set,species,image_name,image_type,grade,vector,unique_identifier,user_id,deleted,last_editor,data_set_name,version,width,height,scales
0,2309984,3627,251,Human,2702_20 BB Human BAL-001.svs,Turnbull,0,"{'x1': 18095, 'x2': 18249, 'y1': 19519, 'y2': ...",e1776329-eb06-486e-bc08-1eb55c9df633,1,False,1,SDATA,Screened,154,152,1.013158
1,2309988,3627,251,Human,2702_20 BB Human BAL-001.svs,Turnbull,0,"{'x1': 16350, 'x2': 16475, 'y1': 10500, 'y2': ...",d54087bb-5634-46ba-81e7-6d9440321cf8,1,False,1,SDATA,Screened,125,115,1.086957
2,2309989,3627,251,Human,2702_20 BB Human BAL-001.svs,Turnbull,0,"{'x1': 14468, 'x2': 14618, 'y1': 22371, 'y2': ...",77c564d3-a151-4658-87d6-7bf74c74f7d0,1,False,1,SDATA,Screened,150,149,1.006711
3,2309990,3627,251,Human,2702_20 BB Human BAL-001.svs,Turnbull,0,"{'x1': 18821, 'x2': 18965, 'y1': 22047, 'y2': ...",5c537e27-4967-47e3-8a0a-5be32f0f4a6d,1,False,1,SDATA,Screened,144,144,1.000000
4,2309992,3627,251,Human,2702_20 BB Human BAL-001.svs,Turnbull,0,"{'x1': 7846, 'x2': 7990, 'y1': 20329, 'y2': 20...",ba0a81f7-8122-45b2-bde1-3980b2485a3a,1,False,1,SDATA,Screened,144,144,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54575,1933562,3614,251,Human,11480_19 humane BAL Berliner Blau.svs,Prussian,3,"{'x1': 12407, 'x2': 12523, 'y1': 14277, 'y2': ...",6420f133-49b0-424e-b3c4-a4061e877571,1,False,1,SDATA,Screened,116,116,1.000000
54576,1935049,3614,251,Human,11480_19 humane BAL Berliner Blau.svs,Prussian,3,"{'x1': 12285, 'x2': 12416, 'y1': 15108, 'y2': ...",e534d92e-faad-4e0f-b570-1950c6e41b01,1,False,1,SDATA,Screened,131,130,1.007692
54577,1936265,3614,251,Human,11480_19 humane BAL Berliner Blau.svs,Prussian,3,"{'x1': 11038, 'x2': 11171, 'y1': 9964, 'y2': 1...",f81bc55c-3ef8-4530-b6b0-87f68b4ec6d2,1,False,1,SDATA,Screened,133,133,1.000000
54578,1936514,3614,251,Human,11480_19 humane BAL Berliner Blau.svs,Prussian,3,"{'x1': 12064, 'x2': 12214, 'y1': 18239, 'y2': ...",656b5b04-f2a1-47b5-9ce6-40181b32e202,1,False,1,SDATA,Screened,150,149,1.006711


## Global Stats

In [12]:
def get_global_stats(frame, dataset , version, species):
    
    nr_slides = len(frame["image_name"].unique())
    total = frame.shape[0]
    score = int(frame["grade"].mean() * 100)
    _0 = frame[frame["grade"] == 0].shape[0]
    _1 = frame[frame["grade"] == 1].shape[0]
    _2 = frame[frame["grade"] == 2].shape[0]
    _3 = frame[frame["grade"] == 3].shape[0]
    _4 = frame[frame["grade"] == 4].shape[0]
    mean = frame["grade"].mean()
    SD = frame["grade"].std()
    
    frame = [[species, dataset, nr_slides, version, total, score, _0, _1, _2, _3, _4, mean, SD]]
    
    return pd.DataFrame(frame, columns=["species", "dataset" ,"slides", "version", "total", "score", "0", "1", "2", "3", "4", "mean", "SD"])

In [13]:
v0_meta = get_global_stats(v0, "SDATA", "Inference", species)
v_cluster_meta = get_global_stats(v_cluster, "SDATA", "Cluster", species)
v_screening_meta = get_global_stats(v_screening, "SDATA", "Screening", species)
v_density_meta = get_global_stats(v_density, "SDATA", "Density", species)

metaDf = pd.concat([v0_meta, v_cluster_meta, v_screening_meta, v_density_meta])
metaDf.head()

,species,dataset,slides,version,total,score,0,1,2,3,4,mean,SD
0,Human,SDATA,12,Inference,168411,133,31035,64833,58320,12776,1447,1.339515,0.892238
0,Human,SDATA,12,Cluster,128012,133,21532,53704,42553,8932,1291,1.334016,0.871160
0,Human,SDATA,12,Screening,54580,156,4726,20688,23323,5090,753,1.568633,0.829320
0,Human,SDATA,12,Density,53864,158,4384,18357,26563,4433,127,1.583432,0.764576


## Stain

In [14]:
v0["image_name"].unique()

array(['2702_20 BB Human BAL-001.svs', '2707_20 TB BAL Human neu.svs',
       '10052_19 humane BAL Berliner Blau-001.svs',
       '10080_19 humane BAL Berliner Blau-001.svs',
       '10120_19 humane BAL Berliner Blau.svs',
       '10227_19 humane BAL Berliner Blau.svs',
       '10672_19 humane BAL Berliner Blau.svs',
       '10672_19 humane BAL Turnbull Blau.svs',
       '10835_19 humane BAL Berliner Blau.svs',
       '10835_19 humane BAL Turnbull Blau.svs',
       '11323_19 humane BAL Berliner Blau.svs',
       '11480_19 humane BAL Berliner Blau.svs'], dtype=object)

In [15]:
"BerlinerBlaue: {}".format(len([name for name in v0["image_name"].unique() if ("erliner" or "BB") in name]))

'BerlinerBlaue: 8'

In [16]:
"TurnbullBlaue: {}".format(len([name for name in v0["image_name"].unique() if ("erliner" or "BB") not in name]))

'TurnbullBlaue: 4'

## Introduced new cells?

In [17]:
v_screening[v_screening["user_id"]==12].shape

(192, 17)

## Images

In [18]:
v0["image_name"].unique()

array(['2702_20 BB Human BAL-001.svs', '2707_20 TB BAL Human neu.svs',
       '10052_19 humane BAL Berliner Blau-001.svs',
       '10080_19 humane BAL Berliner Blau-001.svs',
       '10120_19 humane BAL Berliner Blau.svs',
       '10227_19 humane BAL Berliner Blau.svs',
       '10672_19 humane BAL Berliner Blau.svs',
       '10672_19 humane BAL Turnbull Blau.svs',
       '10835_19 humane BAL Berliner Blau.svs',
       '10835_19 humane BAL Turnbull Blau.svs',
       '11323_19 humane BAL Berliner Blau.svs',
       '11480_19 humane BAL Berliner Blau.svs'], dtype=object)

In [19]:
len(v0["image_name"].unique())

12